# 가설검정

## 귀무가설과 대립가설
- 귀무가설(H0): 효과/차이 없음
- 대립가설(H1): 효과/차이 있음

## 검정 결과 해석
- 검정통계량: 데이터가 H0와 얼마나 다른지 나타내는 값
- p-value (예: α=0.05)
  - p < α  -> H0 기각 (H1 채택)
  - p ≥ α -> H0 기각 불가 (H0 채택)

## 가설 검정 프로세스
1) 가설 설정: H0, H1
2) 유의수준 결정: α=0.05 등
3) 검정통계량 및 p-value 계산
4) 결과 도출 및 해석

# t-검정(t-test) 개요

## 유형 요약
| 검정 유형            | 표본/모집단 구조        | 비교 관계        | 예시                         |
|---------------------|-------------------------|------------------|------------------------------|
| 단일 표본 t-검정     | 모집단 1, 표본 1        | 한 그룹 vs 기준값 | 과자 무게가 200g과 다른지    |
| 대응(쌍체) t-검정    | 같은 집단 전후/짝 자료  | 같은 그룹(쌍)     | 신약 전후 효과               |
| 독립 표본 t-검정     | 서로 다른 두 집단       | 서로 다른 그룹    | 1반 vs 2반 성적 차이         |

## 단일 표본 t-검정 (One-sample)
- 정규성 검정: shapiro(data)
  - Yes -> ttest_1samp(data, popmean)
  - No  -> wilcoxon(data - popmean)

## 대응(쌍체) t-검정 (Paired)
- diff = data1 - data2, 정규성: shapiro(diff)
  - Yes -> ttest_rel(data1, data2)
  - No  -> wilcoxon(data1, data2)

## 독립 표본 t-검정 (Independent)
- 정규성: shapiro(data1), shapiro(data2)
  - Yes -> 등분산: levene(data1, data2)
      - 등분산 Yes -> ttest_ind(data1, data2)
      - 등분산 No  -> ttest_ind(data1, data2, equal_var=False)   # Welch
  - No  -> mannwhitneyu(data1, data2)

## SciPy 함수 요약
- shapiro(x), levene(x,y)
- ttest_1samp(x, popmean)
- ttest_rel(x, y)
- ttest_ind(x, y, equal_var=True|False)
- wilcoxon(x, y) 또는 wilcoxon(x - y)
- mannwhitneyu(x, y)


# 귀무가설: "합격 원두" 상품의 평균 무게는 120g이다.
# 대립가설: "합격 원두" 상품의 평균 무게는 120g이 아니다.
import pandas as pd
df = pd.DataFrame({
    '무게':[119,121,121,119,125,115,121,118,117,127,
             123,129,119,124,114,126,122,124,121,116,
             123,123,127,118,122,117,124,125,123,121],
})

In [19]:
#단일 표본 검정은 "데이터는 정규분포를 따른다"라는 조건이 있을때 진행한다
#단일 표본 검정
from scipy import stats
print(stats.ttest_1samp(df['무게'],120))

TtestResult(statistic=2.153709967150663, pvalue=0.03970987897788578, df=29)


In [21]:
#대립가설  : "합격 원두" 상품의 평균 무게는 120g이 아니다.
print(stats.ttest_1samp(df['무게'],120, alternative='two-sided'))
#=> 귀무가설 기각

TtestResult(statistic=2.153709967150663, pvalue=0.03970987897788578, df=29)


In [23]:
#대립가설  : "합격 원두" 상품의 평균 무게는 120g보다 크다.
print(stats.ttest_1samp(df['무게'],120, alternative='greater'))
#=> 귀무가설 기각

TtestResult(statistic=2.153709967150663, pvalue=0.01985493948894289, df=29)


In [25]:
#대립가설  : "합격 원두" 상품의 평균 무게는 120g보다 작다.
print(stats.ttest_1samp(df['무게'],120, alternative='less'))
#=> 귀무가설 채택

TtestResult(statistic=2.153709967150663, pvalue=0.9801450605110571, df=29)


In [31]:
# 데이터 (정규성에 만족하지 않게 일부 변경)
import pandas as pd
df = pd.DataFrame({
    '무게':[219,121,121,119,125,115,121,118,117,127,
             123,129,119,124,114,126,122,124,121,116,
             123,123,127,118,122,117,124,125,123,121],
})

## Shapiro-Wilk 검정
- 귀무가설(H0) : 주어진 데이터 샘플은 정규 분포를 따른다
- 대립가설(H1) : 주어진 데이터 샘플은 정규 분포를 따르지 않는다

=> 여기서는 귀무가설이 채택되길 바라야 함!!

In [35]:
#shapiro-wilk 정규성 검정
from scipy import stats
stats.shapiro(df['무게'])
#지수표기법으로 되어있음. 소수점이 앞으로 10칸 움직이면 pvalue값이 0.05보다 작기 때문엥 정규분포를 따르지 않는다. (대립가설 채택)

ShapiroResult(statistic=0.35728970196526855, pvalue=2.2139240997414947e-10)

## Wilcoxon 검정
- 귀무가설(H0): "합격 원두" 상품의 평균 무게는 120g이다.
- 대립가설(H1): "합격 원두" 상품의 평균 무게는 120g보다 작다.

In [38]:
#Wilcoxon(윌콕슨)의 부호 순위 검정 수행
stats.wilcoxon(df['무게']-120, alternative='less')

WilcoxonResult(statistic=341.0, pvalue=0.9882949283346534)

# 대응 표본 검정
동일한 집단에 대해서 두 번 측정한 결과를 비교할 때 사용
ex) 약물의 효과, 교육 프로그램의 효과.. 사후 데이터 비교

(데이터는 정규분포를 따른다고 가정)
α = (before - after)의 평균

- 귀무가설 : α >= 0, 새로운 교육 프로그램은 효과가 없다.
- 귀무가설 : α < 0, 새로운 교육 프로그램은 효과가 있다. 

In [41]:
import pandas as pd
df = pd.DataFrame({
    'before':[80,90,92,88,86,89,83,87,65,50],
    'after':[88,89,96,89,85,88,85,89,78,61]
})

In [43]:
#대응표본검정
from scipy import stats
stats.ttest_rel(df['before'],df['after'],alternative='less')
#alternative 넣을 때는 첫번째 값 기준으로 생각

TtestResult(statistic=-2.338738328607322, pvalue=0.022055667780336068, df=9)

α = (after - before)의 평균

- 귀무가설 : α >= 0
- 귀무가설 : α > 0

In [46]:
from scipy import stats
stats.ttest_rel(df['after'],df['before'],alternative='greater')
# after와 before 의 순서를 잘못 적었다면 pvalue값은 똑같고, 검정통계량값만 부호가 다름

TtestResult(statistic=2.338738328607322, pvalue=0.022055667780336068, df=9)

In [48]:
#앞에서는 정규분포를 따른다고 했는데 만약 그런 말이 없다면?
# 정규성을 만족하는지부터 점검
# shapiro-wilk 통해서 검정 가능
import pandas as pd
df = pd.DataFrame({
    'before':[80,90,92,88,86,89,83,87,65,50],
    'after':[88,89,96,89,85,88,85,89,78,61]
})

In [50]:
#Shafiro-Wilk 정규성 검정
#대응표본검정에서는 after-before에 대해서 정규성 검정
df['diff']=df['after']-df['before']
from scipy import stats
stats.shapiro(df['diff'])

#정규성 검정에서 귀무가설 : 정규 분포를 따른다 / 대립가설 : 정규 분포를 따르지 않는다

ShapiroResult(statistic=0.8579010227454918, pvalue=0.07207684733095153)

In [53]:
#Wilcoxon 검정
stats.wilcoxon(df['after'],df['before'],alternative='greater')

WilcoxonResult(statistic=47.5, pvalue=0.0244140625)

# 독립 표본 검정
두 그룹의 차이를 통계적으로 검증
- 귀무가설(H0): 그룹별 시험 평균 점수는 차이가 없다.
- 대립가설(H1) : 그룹별 시험 평균 점수는 차이가 있다.

(데이터는 정규분포를 따르고 분산이 동일하다고 가정한다.)

In [58]:
#대응표본검정에서는 사전,사후이기 때문에 길이가 같아야 하지만 독립 표본 검정에서는 길이가 다를 수 있다
A=[85,90,92,88,86,89,83,87,84,50,60,39,28,48,38,28]
B=[82,82,88,85,84,74,79,69,78,76,85,84,79,89]

In [60]:
#독립 표본 검정
from scipy import stats
stats.ttest_ind(A,B) #(처리집단, 대조집단)

#0.05보다 작기 때문에 대립가설 채택.

TtestResult(statistic=-2.051813915505951, pvalue=0.04964542271174967, df=28.0)

In [63]:
#두 집단의 분산이 다르다(equal_var=False)
stats.ttest_ind(A,B,equal_var=False)

TtestResult(statistic=-2.1837307810153024, pvalue=0.04352730399590312, df=16.729279968729678)

- 귀무가설(H0): 그룹별 시험 평균 점수는 차이가 없다
- 대립가설(H1): B그룹 시험 평균 점수가 더 높다

In [67]:
stats.ttest_ind(A,B,equal_var=True,alternative='less')

TtestResult(statistic=-2.051813915505951, pvalue=0.024822711355874834, df=28.0)

- 귀무가설(H0): 그룹별 시험 평균 점수는 차이가 없다
- 대립가설(H1): A그룹 시험 평균 점수가 더 높다

In [73]:
stats.ttest_ind(A,B,equal_var=True,alternative='greater')

TtestResult(statistic=-2.051813915505951, pvalue=0.9751772886441252, df=28.0)

In [85]:
#Shapiro-Wilk 정규성 검정
from scipy import stats
print(stats.shapiro(A))
stats.shapiro(B)

ShapiroResult(statistic=0.8128696504554708, pvalue=0.00405770158688692)


ShapiroResult(statistic=0.9610027583368682, pvalue=0.739578582064332)

In [89]:
#Levene(래빈) 등분산 검정
stats.levene(A,B)

LeveneResult(statistic=8.013957643762076, pvalue=0.008497116974003)

# Mann-Whitney U 검정(비모수 검정)
- 귀무가설(H0): 그룹별 시험 평균 점수는 차이가 없다
- 대립가설(H1): B그룹 시험 평균 점수가 더 높다

In [93]:
stats.ttest_ind(A,B,equal_var=False,alternative='less')

TtestResult(statistic=-2.1837307810153024, pvalue=0.02176365199795156, df=16.729279968729678)

In [83]:
#Mann-Whityney U(만 휘트니 유 검정)
stats.mannwhitneyu(A,B,alternative='less')

MannwhitneyuResult(statistic=106.0, pvalue=0.40944636368515097)